In [2]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict
import string

import re


In [43]:
popular_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/popular.json")
assert len(popular_examples) == 1
popular_examples = popular_examples[0]

random_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/random.json")
assert len(random_examples) == 1
random_examples = random_examples[0]

recent_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/recent.json")
assert len(recent_examples) == 1
recent_examples = recent_examples[0]

ripple_edits_examples = recent_examples #  +  # random_examples #  + random_examples
len(ripple_edits_examples)

1948

In [44]:
0.15 * len(ripple_edits_examples)

292.2

In [45]:
non_zero_outerloop_count = 0
strong_meta_examples = []
weak_meta_examples = []
for example in ripple_edits_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    # non_zero_outerloop_count += len(outerloop_instances) > 0
    if len(outerloop_instances) > 0 and len(locality_instances) > 0:
        
        efficacy_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        efficacy_queries = [q for q in efficacy_queries if len(q["answers"]) > 0]
        efficacy_queries = [q for q in efficacy_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in efficacy_queries])
        
        if len(efficacy_queries) == 0:
            continue
        
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        strong_meta_examples.append(example)
    elif len(locality_instances) == 0:
        
        efficacy_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        efficacy_queries = [q for q in efficacy_queries if len(q["answers"]) > 0]
        efficacy_queries = [q for q in efficacy_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in efficacy_queries])
        
        if len(efficacy_queries) == 0:
            continue
        
        weak_meta_examples.append(example)
    else:
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        weak_meta_examples.append(example)
        
print("# strong examples:", len(strong_meta_examples))
print("# weak examples:", len(weak_meta_examples))
print("# invalid:", len(ripple_edits_examples) - len(strong_meta_examples) - len(weak_meta_examples))

# strong examples: 1318
# weak examples: 575
# invalid: 55


In [47]:
200 / len(ripple_edits_examples)

0.1026694045174538

In [48]:
n_test = 200
n_valid = 200
np.random.shuffle(strong_meta_examples)
np.random.shuffle(weak_meta_examples)

test_examples = strong_meta_examples[:n_test]
valid_examples = strong_meta_examples[n_test:n_test+n_valid]
train_examples = strong_meta_examples[n_test+n_valid:] + weak_meta_examples
np.random.shuffle(train_examples)

In [51]:
# subset = "recent"

# os.makedirs(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}", exist_ok=True)

In [52]:
# io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/train.jsonl")
# io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/valid.jsonl")
# io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/test.jsonl")

### merge the train/dev/test for all subsets

In [115]:
train_examples = []
valid_examples = []
test_examples = []
for subset in ["popular", "recent", ]:
    train_examples += io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/train.jsonl")
    valid_examples += io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/valid.jsonl")
    test_examples += io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/test.jsonl")
print("# train examples:", len(train_examples))
print("# valid examples:", len(valid_examples))
print("# test examples:", len(test_examples))

# train examples: 2170
# valid examples: 300
# test examples: 300


In [ ]:
# os.makedirs(f"{vars.DATA_DIR}/ripple_edits/meta_train/recent+popular", exist_ok=True)
# io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/recent+popular/train.jsonl")
# io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/recent+popular/valid.jsonl")
# io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/recent+popular/test.jsonl")

In [ ]:
# recent_popular_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train.jsonl")

In [ ]:
# random_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_random/train.jsonl")

In [ ]:
# io.dump_jsonlines(recent_popular_train + random_train, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train_w_random.jsonl")

### Check spurious correlation in dataaset (answer verbtaim in edit)

In [139]:
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/recent/train.jsonl")

In [140]:
len(train_examples)

1493

In [141]:
non_zero_outerloop_count = 0
meta_examples = []
propagation2substring_count = defaultdict(int)
propagation2verbtaim_count = defaultdict(int)
propagation2count = defaultdict(int)
# strong_meta_examples = []
# weak_meta_examples = []
count_rel = defaultdict(int)
count_prop_types = defaultdict(int)

for example in train_examples[:]:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    example["edit"]["text"] = example["edit"]["prompt"]
    meta_examples.append(example["edit"])
    
    outerloop_queries = []
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    propagation2count[k] += 1
                    propagation2verbtaim_count[k] += int(ans_candidates[0] in example["edit"]["text"])
                    propagation2substring_count[k] += int(any(a in q["prompt"] for a in ans_candidates))
                    outerloop_queries.append(q)
    for outer_q in outerloop_queries:
        count_prop_types["efficacy::"+ outer_q["question_type"]] += 1
        count_rel["efficacy::"+ outer_q["relation"]] += 1
    # assert len(outerloop_queries) > 0
    meta_examples.extend(outerloop_queries)
    
    locality_queries = []
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    propagation2count[k] += 1
                    propagation2verbtaim_count[k] += int(ans_candidates[0] in example["edit"]["text"])
                    propagation2substring_count[k] += int(any(a in q["prompt"] for a in ans_candidates))
                    locality_queries.append(q)
    for loc_q in locality_queries:
        count_prop_types["specificity::"+ loc_q["question_type"]] += 1
        count_rel["specificity::"+ loc_q["relation"]] += 1
    
    meta_examples.extend(locality_queries)
    
    # assert len(locality_queries) > 0
        
# print("# strong examples", len(strong_meta_examples))
# print("# weak examples", len(weak_meta_examples))

In [143]:
propagation2count

defaultdict(int,
            {'Logical_Generalization': 813,
             'Compositionality_I': 7037,
             'Subject_Aliasing': 892,
             'Relation_Specificity': 5509,
             'Forgetfulness': 951,
             'Compositionality_II': 751})

In [129]:
{k: round(propagation2verbtaim_count[k] / propagation2count[k] * 100, 1) for k in propagation2count}

{'Subject_Aliasing': 100.0,
 'Relation_Specificity': 1.1,
 'Logical_Generalization': 58.5,
 'Compositionality_I': 2.9,
 'Forgetfulness': 0.0,
 'Compositionality_II': 100.0}

In [32]:
119- 81

38

In [30]:
propagation2substring_count

defaultdict(int,
            {'Logical_Generalization': 9,
             'Relation_Specificity': 8,
             'Compositionality_I': 28,
             'Compositionality_II': 10,
             'Subject_Aliasing': 1,
             'Forgetfulness': 81})

In [29]:
{k: round(propagation2substring_count[k] / propagation2count[k] * 100, 1) for k in propagation2count}

{'Logical_Generalization': 7.4,
 'Relation_Specificity': 0.9,
 'Compositionality_I': 2.8,
 'Compositionality_II': 9.4,
 'Subject_Aliasing': 0.3,
 'Forgetfulness': 68.1}

In [106]:
split = "test"
ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_w_prefix.hf")

In [107]:
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}.jsonl")

In [108]:
len(ds)
len(test_data)

500

In [109]:
ds

Dataset({
    features: ['prompt', 'subject_id', 'relation', 'target_id', 'original_fact', 'context', 'paraphrase', 'object', 'subject'],
    num_rows: 500
})

In [110]:
new_test_data = []

count = 0
for i in range(len(test_data)):
    original_test = test_data[i]
    augmentation = ds[i]
    # if original_test["edit"]["prompt"] == "The name of the composer of Klavierstücke  I–IV is Karlheinz Stockhausen.":
    #     # print("found")
    #     original_test["edit"]["subject"] = "name of the composer of Klavierst\u00fccke  I\u2013IV"
    #     original_test["edit"]["object"] = "Karlheinz Stockhausen"
    #     assert original_test["edit"]["subject"] in original_test["edit"]["prompt"]
    #     assert original_test["edit"]["object"] in original_test["edit"]["prompt"]
    #     new_test_data.append(original_test)
    #     # count += 1
    #     continue
    #     # print(original_test)
    #     # print(augmentation)
    # else:
        
    assert original_test["edit"]["prompt"] == augmentation["prompt"], f"{i}@@" + original_test["edit"]["prompt"] + "@@" + augmentation["prompt"]
    assert augmentation["subject"] in original_test["edit"]["prompt"], f"{i}@@" + original_test["edit"]["prompt"] + "@@" + augmentation["subject"]
    assert augmentation["object"] in original_test["edit"]["prompt"]

    original_test["edit"]["subject"] = augmentation["subject"]
    original_test["edit"]["object"] = augmentation["object"]
    new_test_data.append(original_test)
    

In [111]:
new_test_data[0]

{'example_type': 'popular',
 'edit': {'prompt': 'The name of the continent which United Arab Emirates is part of is Indian Ocean.',
  'subject_id': 'Q878',
  'relation': 'CONTINENT',
  'target_id': 'Q1239',
  'original_fact': {'prompt': 'The name of the continent which United Arab Emirates is part of is Asia.',
   'subject_id': 'Q878',
   'relation': 'CONTINENT',
   'target_id': 'Q48'},
  'subject': 'The name of the continent',
  'object': 'Indian Ocean.'},
 'Logical_Generalization': [],
 'Compositionality_I': [],
 'Compositionality_II': [{'test_queries': [{'prompt': 'The name of the continent which the country Strait of Hormuz is associated with is part of is',
     'answers': [{'value': 'Indian Ocean',
       'aliases': ['Eastern Ocean',
        'Eastern Sea',
        'Indian Sea',
        'Oriental Sea',
        'Oriental Ocean']}],
     'query_type': 'two_hop',
     'subject_id': 'Q79883',
     'relation': 'COUNTRY',
     'target_ids': ['Q878'],
     'phrase': 'The name of the cont

In [100]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")


### Prepare a ripple edits for MEND-original version (before generating paraphrase)

In [3]:
split = "test"
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}.jsonl")
train_ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_w_prefix.hf")

# special treatment for test set

In [4]:

new_examples = []
missing_examples = []
mismatch_c = 0
for i in range(len(train_examples)):
    augmentation = train_ds[i-mismatch_c]
    example = train_examples[i]
    new_example = deepcopy(example)
    
    new_example["edit"]["context"] = augmentation["context"]
    new_example["edit"]["paraphrase"] = augmentation["paraphrase"]
    new_example["edit"]["object"] = augmentation["object"]
    new_examples.append(new_example)
    # if example["edit"]["prompt"] != augmentation["prompt"]:
    #     mismatch_c += 1
    # else:
        
    #     assert example["edit"]["prompt"] == augmentation["prompt"]
    #     assert augmentation["object"] in example["edit"]["prompt"]
    
    #     flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["object"], "paraphrase": augmentation["paraphrase"]})
    

In [ ]:
# if split == "test":
#     io.dump_jsonlines(new_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_mend.jsonl")

In [88]:
flattend_train_examples = []
missing_examples = []
mismatch_c = 0
for i in range(len(train_examples)):
    augmentation = train_ds[i-mismatch_c]
    example = train_examples[i]
    
    if example["edit"]["prompt"] != augmentation["prompt"]:
        mismatch_c += 1
    else:
        
        assert example["edit"]["prompt"] == augmentation["prompt"]
        assert augmentation["object"] in example["edit"]["prompt"], augmentation["object"] + "@@@" + example["edit"]["prompt"]
    
        flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["object"], "paraphrase": augmentation["paraphrase"]})
    
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    assert "phrase" in q
                    if q["phrase"] is not None:
                        flattend_train_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "paraphrase": q["phrase"]})
                    else:
                        missing_examples.append({"context": q["prompt"], "completion": ans_candidates[0],})
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    
                    assert "phrase" in q
                    if q["phrase"] is not None:
                        flattend_train_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "paraphrase": q["phrase"]})
                    else:
                        missing_examples.append({"context": q["prompt"], "completion": ans_candidates[0],})
    

In [74]:
# missing_ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/{split}_w_paraphrase.hf")
# for i in range(len(missing_ds)):
#     augmentation = missing_ds[i]
    
#     flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["completion"], "paraphrase": augmentation["paraphrase"]})

In [89]:
len(flattend_train_examples)

17096

In [ ]:
# io.dump_jsonlines(flattend_train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_mend.jsonl")

In [78]:
for ex in io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/valid_mend.jsonl"):
    assert "context" in ex and ex["context"] is not None
    assert "completion" in ex and ex["completion"] is not None
    assert "paraphrase" in ex and ex["paraphrase"] is not None

In [11]:
converted_recoe_aggregation[-1]
len(converted_recoe_aggregation)

508

### Integrate RoCE data to ripple edit trainings

In [116]:
recoe_aggregation = io.load_json("/u/zliu/datastor1/ReCoE/data/aggregation/counterfactual_datapoints_verified_atomic.json")

converted_recoe_aggregation = [
    {
        "texts": " ".join([f["fact"] for f in x["direct_counterfactual_fact"]]),
        "question": x["question"],
        "answer": x["counterfactual_answer"]
     }
    for x in recoe_aggregation
]
assert len(converted_recoe_aggregation) == len(recoe_aggregation)


In [117]:
recoe_comparative = io.load_json("/u/zliu/datastor1/ReCoE/data/comparative/counterfactual_datapoints_verified_atomic.json")
converted_recoe_comparative = []
for x in recoe_comparative:
    assert "choice_1_counterfactuals" in x["counterfactuals_per_choice"]
    assert "choice_2_counterfactuals" in x["counterfactuals_per_choice"]
    assert "choice_3_counterfactuals" not in x["counterfactuals_per_choice"]
    
    counter_fact1_text = [f["fact"] for f in x["counterfactuals_per_choice"]["choice_1_counterfactuals"]]
    counter_fact2_text = [f["fact"] for f in x["counterfactuals_per_choice"]["choice_2_counterfactuals"]]
    texts = [counter_fact1_text, counter_fact2_text]
    np.random.shuffle(texts)
    counter_fact1_text_after_rand, counter_fact2_text_after_rand = texts
    
    
    converted_recoe_comparative.append(
        {
            "texts": " ".join(counter_fact1_text_after_rand + counter_fact2_text_after_rand),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(converted_recoe_comparative) == len(recoe_comparative)

In [118]:
# [len(x["facts_per_choice"]) for x in recoe_comparative]
recoe_counting = io.load_json("/u/zliu/datastor1/ReCoE/data/counting/counterfactual_datapoints_verified_atomic.json")
converted_recoe_counting = []
for x in recoe_counting:
    converted_recoe_counting.append(
        {
            "texts": " ".join([f["sentence"] for f in x["counterfactual_facts"]]),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(recoe_counting) == len(converted_recoe_counting)

In [119]:
recoe_sorting = io.load_json("/u/zliu/datastor1/ReCoE/data/sorting/counterfactual_datapoints_verified_atomic.json")
converted_recoe_sorting = []
for x_i, x in enumerate(recoe_sorting):
    assert "counterfactuals_per_choice" in x
    n_facts = len([k for k in x["facts_per_choice"].keys() if "choice_" in k])
    texts = []
    count = 0
    # assert n_facts == 3
    counter_factuals = x["counterfactuals_per_choice"]
    factuals = x["facts_per_choice"]
    
    unchanged_facts_added = False
    for f_i in range(1, n_facts + 1):
        
        if f"choice_{f_i}_counterfactuals" not in counter_factuals:
            if not unchanged_facts_added:
                texts.extend(counter_factuals["unchanged_facts"])
            else:
                continue
        else:
            texts.extend([f["fact"] for f in counter_factuals[f"choice_{f_i}_counterfactuals"]])
    converted_recoe_sorting.append(
        {
            "texts": " ".join(texts),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(converted_recoe_sorting) == len(recoe_sorting)

In [120]:
recoe_subtraction = io.load_json("/u/zliu/datastor1/ReCoE/data/subtraction/counterfactual_datapoints_verified_atomic.json")


converted_recoe_subtraction = [
    {
        "texts": " ".join([f["fact"] for f in x["direct_counterfactual_fact"]]),
        "question": x["question"],
        "answer": x["counterfactual_answer"]
     }
    for x in recoe_subtraction
]
assert len(recoe_subtraction) == len(converted_recoe_subtraction)


In [ ]:
recoe_superlative = io.load_json("/u/zliu/datastor1/ReCoE/data/superlative/counterfactual_datapoints_verified_atomic.json")

converted_recoe_superlative = []
for x_i, x in enumerate(recoe_superlative):
    assert "counterfactuals_per_choice" in x
    n_facts = len([k for k in x["facts_per_choice"].keys() if "choice_" in k])
    texts = []
    count = 0
    # assert n_facts == 3
    counter_factuals = x["counterfactuals_per_choice"]
    factuals = x["facts_per_choice"]
    
    unchanged_facts_added = False
    for f_i in range(1, n_facts + 1):
        
        if f"choice_{f_i}_counterfactuals" not in counter_factuals:
            if not unchanged_facts_added:
                texts.extend(counter_factuals["unchanged_facts"])
            else:
                continue
        else:
            texts.extend([f["fact"] for f in counter_factuals[f"choice_{f_i}_counterfactuals"]])
    converted_recoe_superlative.append(
        {
            "texts": " ".join(texts),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(recoe_superlative) == len(converted_recoe_superlative)
    # assert count == 2, x

In [122]:
converted_recoe = converted_recoe_aggregation + converted_recoe_comparative + converted_recoe_counting + converted_recoe_sorting + converted_recoe_subtraction + converted_recoe_superlative

In [130]:
converted_recoe = [
    {
        "texts": x["texts"],
        "question": x["question"],
        "answer": str(x["answer"])
    }
    for x in converted_recoe
]

In [131]:
all([isinstance(x["answer"], str) for x in converted_recoe])

True

In [132]:
io.dump_jsonlines(converted_recoe, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/recoe.jsonl")

In [133]:
recent_popular_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train.jsonl")

In [134]:
io.dump_jsonlines(recent_popular_train + converted_recoe, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train_w_recoe.jsonl")